In [12]:
import tensorflow as tf
import numpy as np

In [13]:
tf.reset_default_graph()
done_mask = tf.placeholder(tf.bool, shape=None)

sess = tf.Session()
sess.run(
    done_mask,
    feed_dict={
        done_mask: 0
    }
)

array(False)

### test linear layer

In [41]:
tf.reset_default_graph()

state = tf.placeholder(tf.float32, shape=[None, 80, 80, 4])
scope = 'q'
reuse=False
num_actions = 5

def add_linear(state, scope, reuse):
    _, img_height, img_width, nchannels = state.shape
    state_flat_dim = img_height * img_width * nchannels
    with tf.variable_scope(scope, reuse=reuse):
        batch_state = tf.reshape(state, [-1, state_flat_dim])
        W = tf.get_variable(
            "weights",
            shape=(state_flat_dim, num_actions),
            initializer=tf.constant_initializer(1.)
        )
        tf.add_to_collection(scope, W)
        b = tf.get_variable(
            "bias",
            shape=(num_actions),
            initializer=tf.constant_initializer(0.)
        )
        tf.add_to_collection(scope, b)
        out = tf.matmul(batch_state, W) + b
    return out


out = add_linear(state, scope, reuse=False)
sess = tf.Session()

sess.run(tf.global_variables_initializer())
output_np = sess.run(
    out,
    feed_dict={
        state: np.ones((2, 80, 80, 4))
    }
)


output_np == np.array([
    [25600., 25600., 25600., 25600., 25600.],
    [25600., 25600., 25600., 25600., 25600.]], dtype=np.float32)

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

### update op

In [387]:
tf.reset_default_graph()
with tf.variable_scope("target_q"):
    for i in range(2):
        W = tf.get_variable(
            "weights_" + str(i),
            shape=(2, 2),
            initializer=tf.constant_initializer(1.)
        )
        tf.add_to_collection('target_q', W)

with tf.variable_scope("q"):
    for i in range(2):
        W = tf.get_variable(
            "weights_" + str(i),
            shape=(2, 2),
            initializer=tf.constant_initializer(2.)
        )
        tf.add_to_collection('q', W)

In [388]:
tf.get_collection('q')

[<tf.Variable 'q/weights_0:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'q/weights_1:0' shape=(2, 2) dtype=float32_ref>]

In [389]:
target_q_var_lst = tf.get_collection('target_q')
q_var_lst = tf.get_collection('q')
update_op_lst = []
for idx, target_q_var in enumerate(target_q_var_lst):
    op = target_q_var.assign(q_var_lst[idx])
    update_op_lst.append(op.op)
update_op_grouped = tf.group(*update_op_lst)

In [391]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

print(sess.run(
    target_q_var_lst,
))

sess.run(update_op_grouped)

print(sess.run(
    target_q_var_lst,
))

print(sess.run(
    q_var_lst,
))

[array([[1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.]], dtype=float32)]
[array([[2., 2.],
       [2., 2.]], dtype=float32), array([[2., 2.],
       [2., 2.]], dtype=float32)]
[array([[2., 2.],
       [2., 2.]], dtype=float32), array([[2., 2.],
       [2., 2.]], dtype=float32)]


### loss

In [62]:
a = tf.constant([0, 1])
q = tf.constant([
    [0.1, 0.2, 0.3],
    [0.6, 0.5, 0.4]   
])
target_q = tf.constant([
    [1.1, 1.2, 1.3],
    [1.6, 1.5, 1.4]   
])
r = tf.constant([1., 10.])
done_mask = tf.constant([0., 1.])
num_actions = 3
gamma = 1.0

In [63]:
a_one_hot = tf.one_hot(a, depth=num_actions)
loss_agg = r + \
(1. - done_mask) * gamma * tf.reduce_max(target_q, axis=1) - \
tf.reduce_sum(tf.multiply(q, a_one_hot), axis=1)
loss = tf.reduce_mean(loss_agg ** 2)

In [75]:
sess = tf.Session()
sess.run(loss)


47.545

In [67]:
a = np.array([0, 1])
q = np.array([
    [0.1, 0.2, 0.3],
    [0.6, 0.5, 0.4]   
])
target_q = np.array([
    [1.1, 1.2, 1.3],
    [1.6, 1.5, 1.4]   
])
r = np.array([1., 10.])
done_mask = np.array([0., 1.])
num_actions = 3
gamma = 1.0

In [72]:
loss_agg = r + (1. - done_mask) * target_q.max(axis=1) - q[[0, 1], a]

In [74]:
np.mean(loss_agg ** 2)

47.545

### optimizer op

In [354]:
tf.reset_default_graph()

s = tf.placeholder(tf.float32, shape=[None, 2, 2, 1])
sp = tf.placeholder(tf.float32, shape=[None, 2, 2, 1])
a = tf.placeholder(tf.int32, shape=None)
r = tf.placeholder(tf.float32, shape=None)
gamma = tf.placeholder(tf.float32, shape=())
done_mask = tf.placeholder(tf.float32, shape=None)

num_actions = 2

def add_linear(state, scope, reuse, init_value):
    _, img_height, img_width, nchannels = state.shape
    state_flat_dim = img_height * img_width * nchannels
    with tf.variable_scope(scope, reuse=reuse):
        batch_state = tf.reshape(state, [-1, state_flat_dim])
        W = tf.get_variable(
            "weights",
            shape=(state_flat_dim, num_actions),
            initializer=tf.constant_initializer(init_value)
        )
        tf.add_to_collection(scope, W)
        b = tf.get_variable(
            "bias",
            shape=(num_actions),
            initializer=tf.constant_initializer(init_value)
        )
        tf.add_to_collection(scope, b)
        out = tf.matmul(batch_state, W) + b
    return out

target_q = add_linear(sp, scope='target_q', reuse=False, init_value=1.)
q = add_linear(s, scope='q', reuse=False, init_value=2.)
a_one_hot = tf.one_hot(a, depth=num_actions)

loss_agg = r + \
  (1. - done_mask) * gamma * tf.reduce_max(target_q, axis=1) - \
    tf.reduce_sum(tf.multiply(q, a_one_hot), axis=1)
loss = tf.reduce_mean(loss_agg ** 2)

In [355]:
scope = 'q'
lr = 0.01
grad_clip = True
clip_val = 1.
var_lst = tf.get_collection(scope)
optimizer = tf.train.GradientDescentOptimizer(lr)
grads_and_vars_lst = optimizer.compute_gradients(
    loss, var_list=var_lst)

# self.train_op = optimizer.minimize(self.loss, var_list=var_lst)

if grad_clip:
    grads_clipped_and_vars_lst = []
    for grad_and_var in grads_and_vars_lst:
        grad, var = grad_and_var
        grad_clipped = tf.clip_by_norm(grad, clip_val)
        grads_clipped_and_vars_lst.append((grad_clipped, var))
    train_op = optimizer.apply_gradients(
        grads_clipped_and_vars_lst)
    grads_lst = [x[0] for x in grads_clipped_and_vars_lst]
else:
    train_op = optimizer.apply_gradients(grads_and_vars_lst)
    grads_lst = [x[0] for x in grads_and_vars_lst]

# global norm is just a norm of stack vectors
grad_norm = tf.global_norm(grads_lst)

In [356]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(
    loss,
    feed_dict={
        s: np.ones((1, 2, 2, 1)),
        a: np.array([1]),
        r: np.array([0.]),
        sp: np.ones((1, 2, 2, 1)),
        done_mask: np.array([1.]),
        gamma: 1.
    }
)

100.0

In [378]:
sess.run(
    [train_op,
     loss,
     grad_norm
     
    ],
    feed_dict={
        s: np.ones((1, 2, 2, 1)),
        a: np.array([0]),
        r: np.array([0.]),
        sp: np.ones((1, 2, 2, 1)),
        done_mask: np.array([1.]),
        gamma: 1.
    }
)

[None, 87.79691, 1.4142135]

In [352]:
q_vars_lst = tf.get_collection('q')

In [353]:
sess.run(q_vars_lst)

[array([[0.0008222 , 0.00751421],
        [0.0008222 , 0.00751421],
        [0.0008222 , 0.00751421],
        [0.0008222 , 0.00751421]], dtype=float32),
 array([0.0008222 , 0.00751421], dtype=float32)]

In [386]:
a = tf.constant([1., 1.])
sess.run(tf.norm([a], ord=1))

2.0

### optimizer op CNN

In [958]:
tf.reset_default_graph()

s = tf.placeholder(tf.float32, shape=[None, 80, 80, 1])
sp = tf.placeholder(tf.float32, shape=[None, 80, 80, 1])
a = tf.placeholder(tf.int32, shape=None)
r = tf.placeholder(tf.float32, shape=None)
gamma = tf.placeholder(tf.float32, shape=())
done_mask = tf.placeholder(tf.float32, shape=None)

num_actions = 2

def add_CNN(state, scope, reuse, init_value):
    with tf.variable_scope(scope, reuse=reuse):

        layer1 = tf.contrib.layers.conv2d(
            inputs=state,
            num_outputs=32,
            kernel_size=[8, 8],
            stride=4,
            padding="SAME",
            activation_fn=tf.nn.relu,
            variables_collections=scope
        )

        layer2 = tf.contrib.layers.conv2d(
            inputs=layer1,
            num_outputs=64,
            kernel_size=[4, 4],
            stride=2,
            padding="SAME",
            activation_fn=tf.nn.relu,
            variables_collections=scope
        )

        layer3 = tf.contrib.layers.conv2d(
            inputs=layer2,
            num_outputs=64,
            kernel_size=[3, 3],
            stride=1,
            padding="SAME",
            activation_fn=tf.nn.relu,
            variables_collections=scope
        )

        layer4 = tf.contrib.layers.fully_connected(
            tf.reshape(layer3, [-1, 10 * 10 * 64]),
            512,
            activation_fn=tf.nn.relu,
            variables_collections=scope
        )

        layer5 = tf.contrib.layers.fully_connected(
            layer4,
            num_actions,
            activation_fn=None,
            variables_collections=scope
        )
        out = layer5
    return out


target_q = add_CNN(sp, scope='target_q', reuse=False, init_value=1.)
q = add_CNN(s, scope='q', reuse=False, init_value=2.)
a_one_hot = tf.one_hot(a, depth=num_actions)

loss_agg = r + \
  (1. - done_mask) * gamma * tf.reduce_max(target_q, axis=1) - \
    tf.reduce_sum(tf.multiply(q, a_one_hot), axis=1)
loss = tf.reduce_mean(loss_agg ** 2)

In [963]:
scope = 'q'
lr = 0.05
grad_clip = True
clip_val = 10.
var_lst = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='q')

In [964]:
var_lst

[<tf.Variable 'q/Conv/weights:0' shape=(8, 8, 1, 32) dtype=float32_ref>,
 <tf.Variable 'q/Conv/biases:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'q/Conv_1/weights:0' shape=(4, 4, 32, 64) dtype=float32_ref>,
 <tf.Variable 'q/Conv_1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'q/Conv_2/weights:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'q/Conv_2/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'q/fully_connected/weights:0' shape=(6400, 512) dtype=float32_ref>,
 <tf.Variable 'q/fully_connected/biases:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'q/fully_connected_1/weights:0' shape=(512, 2) dtype=float32_ref>,
 <tf.Variable 'q/fully_connected_1/biases:0' shape=(2,) dtype=float32_ref>]

In [596]:
optimizer = tf.train.GradientDescentOptimizer(lr)
grads_and_vars_lst = optimizer.compute_gradients(
    loss, var_list=var_lst)

# self.train_op = optimizer.minimize(self.loss, var_list=var_lst)

if grad_clip:
    grads_clipped_and_vars_lst = []
    for grad_and_var in grads_and_vars_lst:
        grad, var = grad_and_var
        grad_clipped = tf.clip_by_norm(grad, clip_val)
        grads_clipped_and_vars_lst.append((grad_clipped, var))
    train_op = optimizer.apply_gradients(
        grads_clipped_and_vars_lst)
    grads_lst = [x[0] for x in grads_clipped_and_vars_lst]
else:
    train_op = optimizer.apply_gradients(grads_and_vars_lst)
    grads_lst = [x[0] for x in grads_and_vars_lst]

# global norm is just a norm of stack vectors
grad_norm = tf.global_norm(grads_lst)

In [597]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(
    loss,
    feed_dict={
        s: np.ones((1, 80, 80, 1)),
        a: np.array([1]),
        r: np.array([0.]),
        sp: np.ones((1, 80, 80, 1)),
        done_mask: np.array([1.]),
        gamma: 1.
    }
)

163891200.0

In [954]:
sess.run(
    [train_op,
     loss,
     grad_norm
     
    ],
    feed_dict={
        s: np.ones((1, 80, 80, 1)),
        a: np.array([0]),
        r: np.array([0.]),
        sp: np.ones((1, 80, 80, 1)),
        done_mask: np.array([1.]),
        gamma: 1.
    }
)

[None, 23.794497, 13.970612]

In [352]:
q_vars_lst = tf.get_collection('q')

In [353]:
sess.run(q_vars_lst)

[array([[0.0008222 , 0.00751421],
        [0.0008222 , 0.00751421],
        [0.0008222 , 0.00751421],
        [0.0008222 , 0.00751421]], dtype=float32),
 array([0.0008222 , 0.00751421], dtype=float32)]

In [386]:
a = tf.constant([1., 1.])
sess.run(tf.norm([a], ord=1))

2.0